# RL For Controlled Environment Agriculture

- i want to make a system that is robust to structural environmental variation
    - eg. buildings of different forms
    - i want it to intuit from the structure and historical data, a reasonable policy
    - given minimal information and minimal sampling, provide a robust system

- learn the physics of the system so that the algorithm can be transported to multiple environments
    - this means that the relevant features of the building need to be incorporated to the state
- Training needs to focus on dealing with stochastically generated environments
    - new ideal temp and environmental conditions between sets of episodes within some range

- using a markov model to take what experiences have been seen and generate more from similar distributions
    - this would address sample efficiency

Related Work


# todo
1. create a continuous version of the greenhouse gym so that we can use policy based methods
    - This will involve scaling the cooling to an action space of [-1, 1]
    - handling the scaling from [-1, 1] to [min_action, max_action] in the environment makes system more robust to changing agents
    - use box2D for the action space
    - re-look at bipedal if confused
2. add radiative heat flow to the problem
3. create mach paper where we examine the behavior of three agents in greenhouse environment
4. clean agent code